In [1]:
using DataFramesMeta
using StatsBase
using CSV

dir = "/Users/fujisetakumi/_private/data_science/100knocks-preprocess/data/"
df_customers  = CSV.read(dir * "customers.csv", DataFrame)
df_categories = CSV.read(dir * "categories.csv", DataFrame)
df_products   = CSV.read(dir * "products.csv", DataFrame)
df_geocodes   = CSV.read(dir * "geocodes.csv", DataFrame)
df_receipts   = CSV.read(dir * "receipts.csv", DataFrame)
df_stores     = CSV.read(dir * "stores.csv", DataFrame)
;
ENV["COLUMNS"] = 1000
;

In [2]:
# P-026
@chain df_receipts begin
    @orderby(:customer_id, :sales_ymd)
    groupby(:customer_id)
    combine(:sales_ymd => first, :sales_ymd => last)
    @subset(:sales_ymd_first .!= :sales_ymd_last)
    first(10)
end

Row,customer_id,sales_ymd_first,sales_ymd_last
,String15,Int64,Int64
1,CS001114000005,20180503,20190731
2,CS001115000010,20171228,20190405
3,CS001205000004,20170914,20190625
4,CS001205000006,20180207,20190224
5,CS001214000009,20170306,20190902
6,CS001214000017,20180828,20191006
7,CS001214000048,20171109,20190929
8,CS001214000052,20180208,20190617
9,CS001215000005,20170206,20181021


In [3]:
# P-027 平均値
@chain df_receipts begin
    groupby(:store_cd)
    combine(:amount => mean)
    sort(:amount_mean, rev=true)
    first(5)
end

Row,store_cd,amount_mean
,String7,Float64
1,S13052,402.867
2,S13015,351.112
3,S13003,350.916
4,S14010,348.791
5,S13001,348.47


In [4]:
# P-028 中央値
@chain df_receipts begin
    groupby(:store_cd)
    combine(:amount => median)
    sort(:amount_median, rev=true)
    first(10)
end

Row,store_cd,amount_median
,String7,Float64
1,S13052,190.0
2,S14010,188.0
3,S14050,185.0
4,S13003,180.0
5,S14040,180.0
6,S13018,180.0
7,S13008,179.0
8,S14033,179.0
9,S14025,178.0


In [5]:
# P-029 最頻値
@chain df_receipts begin
    groupby(:store_cd)
    combine(:product_cd => mode)
    sort(:product_cd_mode, rev=true)
    first(10)
end

Row,store_cd,product_cd_mode
,String7,String15
1,S13051,P080804001
2,S13003,P071401001
3,S13020,P071401001
4,S13039,P071401001
5,S14026,P071401001
6,S13041,P071401001
7,S13015,P071401001
8,S13019,P071401001
9,S13018,P071401001


In [6]:
# P-030 分散
@chain df_receipts begin
    groupby(:store_cd)
    combine(:amount => var)
    sort(:amount_var, rev=true)
    first(10)
end

Row,store_cd,amount_var
,String7,Float64
1,S13052,4.41863e5
2,S14011,3.06442e5
3,S14034,2.97068e5
4,S13001,2.95559e5
5,S13015,2.95427e5
6,S13035,2.78121e5
7,S14045,2.66643e5
8,S14046,2.64461e5
9,S14010,2.58568e5


In [7]:
# P-031 標準偏差
stdpop(x) = std(x, corrected=false)
@chain df_receipts begin
    groupby(:store_cd)
    combine(:amount => stdpop, :amount => std)
    sort(:amount_stdpop, rev=true)
    first(10)
end

Row,store_cd,amount_stdpop,amount_std
,String7,Float64,Float64
1,S13052,663.392,664.728
2,S14011,553.457,553.572
3,S14034,544.904,545.04
4,S13001,543.537,543.653
5,S13015,543.41,543.532
6,S13035,527.253,527.372
7,S14045,516.189,516.375
8,S14046,514.065,514.257
9,S14010,508.384,508.496


In [8]:
# P-032 25%刻みパーセンタイル値
using StatsPlots

values = map(i -> quantile(df_receipts.amount, i), [0.25, 0.50, 0.75, 1.00])
display(values)

boxplot(df_receipts.amount)

4-element Vector{Float64}:
   102.0
   170.0
   288.0
 10925.0

In [9]:
# P-033 平均
@chain df_receipts begin
    groupby(:store_cd)
    combine(:amount => mean)
    @subset(:amount_mean .> 330)
    sort(:store_cd)
end

Row,store_cd,amount_mean
,String7,Float64
1,S12013,330.194
2,S13001,348.47
3,S13003,350.916
4,S13004,330.944
5,S13015,351.112
6,S13019,330.209
7,S13020,337.88
8,S13052,402.867
9,S14010,348.791


In [10]:
# P-034 顧客ごとの売上金額の平均
@chain df_receipts begin
    @rsubset(occursin(r"^[^Z]", :customer_id))
    groupby(:customer_id)
    combine(:amount => sum)
    combine(:amount_sum => mean)
end

Row,amount_sum_mean
,Float64
1,2547.74


In [11]:
# P-035 平均売上額を上回る顧客
summary = @chain df_receipts begin
    @rsubset(occursin(r"^[^Z]", :customer_id))
    groupby(:customer_id)
    combine(:amount => sum)
end

amount_sum_mean = mean(summary.amount_sum)

@chain summary begin
    @rsubset(:amount_sum > amount_sum_mean)
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum
,String15,Int64
1,CS001115000010,3044
2,CS001205000006,3337
3,CS001214000009,4685
4,CS001214000017,4132
5,CS001214000052,5639
6,CS001215000040,3496
7,CS001304000006,3726
8,CS001305000005,3485
9,CS001305000011,4370


In [12]:
# P-036
@chain df_receipts begin
    innerjoin(df_stores, on= :store_cd => :store_cd)
    sort(:store_cd)
    first(10)
end

Row,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,Int64,Int64,String7,Int64,Int64,String15,String15,Int64,Int64,String31,Int64,String15,String,String,String15,Float64,Float64,Float64
1,20190221,1550707200,S12007,1132,2,CS007513000235,P080104025,1,180,佐倉店,12,千葉県,千葉県佐倉市上志津,チバケンサクラシカミシヅ,043-123-4001,140.145,35.7187,1895.0
2,20170706,1499299200,S12007,1142,1,CS007615000066,P071401011,1,1300,佐倉店,12,千葉県,千葉県佐倉市上志津,チバケンサクラシカミシヅ,043-123-4001,140.145,35.7187,1895.0
3,20180904,1536019200,S12007,1112,1,CS007515000170,P060503001,1,78,佐倉店,12,千葉県,千葉県佐倉市上志津,チバケンサクラシカミシヅ,043-123-4001,140.145,35.7187,1895.0
4,20170909,1504915200,S12007,1102,2,ZZ000000000000,P070308044,1,548,佐倉店,12,千葉県,千葉県佐倉市上志津,チバケンサクラシカミシヅ,043-123-4001,140.145,35.7187,1895.0
5,20170623,1498176000,S12007,1182,1,CS007512000139,P060102005,1,90,佐倉店,12,千葉県,千葉県佐倉市上志津,チバケンサクラシカミシヅ,043-123-4001,140.145,35.7187,1895.0
6,20171130,1512000000,S12007,1162,1,ZZ000000000000,P091503004,1,1130,佐倉店,12,千葉県,千葉県佐倉市上志津,チバケンサクラシカミシヅ,043-123-4001,140.145,35.7187,1895.0
7,20170524,1495584000,S12007,1122,2,ZZ000000000000,P071005005,1,98,佐倉店,12,千葉県,千葉県佐倉市上志津,チバケンサクラシカミシヅ,043-123-4001,140.145,35.7187,1895.0
8,20170913,1505260800,S12007,1162,1,CS007414000032,P050601003,1,170,佐倉店,12,千葉県,千葉県佐倉市上志津,チバケンサクラシカミシヅ,043-123-4001,140.145,35.7187,1895.0
9,20181222,1545436800,S12007,1192,1,CS007514000086,P040102001,1,268,佐倉店,12,千葉県,千葉県佐倉市上志津,チバケンサクラシカミシヅ,043-123-4001,140.145,35.7187,1895.0


In [13]:
# P-037
@chain df_products begin
    innerjoin(df_categories,
        on= :category_small_cd => :category_small_cd,
        makeunique=true
    )
    select(names(df_products), :category_small_name)
    first(10)
end

Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
,String15,Int64,Int64,Int64,Int64?,Int64?,String
1,P040101001,4,401,40101,198,149,弁当類
2,P040101002,4,401,40101,218,164,弁当類
3,P040101003,4,401,40101,230,173,弁当類
4,P040101004,4,401,40101,248,186,弁当類
5,P040101005,4,401,40101,268,201,弁当類
6,P040101006,4,401,40101,298,224,弁当類
7,P040101007,4,401,40101,338,254,弁当類
8,P040101008,4,401,40101,420,315,弁当類
9,P040101009,4,401,40101,498,374,弁当類


In [14]:
# P-038
women_who_bought = @chain df_customers begin
    @rsubset(:gender_cd == 1, occursin(r"^[^Z]", :customer_id))
    leftjoin(df_receipts, on= :customer_id => :customer_id)
    groupby(:customer_id)
    combine(:amount => sum)
    @rtransform(:amount_sum = coalesce(:amount_sum, 0))
    sort(:customer_id)
end

@chain df_customers begin
    innerjoin(women_who_bought, on=:customer_id)
    first(10)
end

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,amount_sum
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15,Int64
1,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,5088
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,875
5,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B,3122
6,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5,868
7,CS035614000014,板倉 菜々美,1,女性,1954-07-16,64,154-0015,東京都世田谷区桜新町**********,S13035,20150804,0-00000000-0,0
8,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9,3444
9,CS009413000079,市川 コウ,1,女性,1975-12-28,43,158-0093,東京都世田谷区上野毛**********,S13009,20151209,0-00000000-0,0


In [15]:
# P-039
targets1 = @chain df_receipts begin
    @rsubset(occursin(r"^[^Z]", :customer_id))
    groupby(:customer_id)
    combine(:sales_ymd => length)
    sort(:sales_ymd_length, rev=true)
    first(20)
end

targets2 = @chain df_receipts begin
    @rsubset(occursin(r"^[^Z]", :customer_id))
    groupby(:customer_id)
    combine(:amount => sum)
    sort(:amount_sum, rev=true)
    first(20)
end

@chain targets1 begin
    outerjoin(targets2, on=:customer_id)
    sort(:customer_id)
end

Row,customer_id,sales_ymd_length,amount_sum
,String15,Int64?,Int64?
1,CS001605000009,missing,18925
2,CS006515000023,missing,18372
3,CS007514000094,missing,15735
4,CS007515000107,40,missing
5,CS009414000059,missing,15492
6,CS010214000002,42,missing
7,CS010214000010,44,18585
8,CS011414000106,missing,18338
9,CS011415000006,missing,16094


In [16]:
# P-040
nrow(df_stores) * nrow(df_products)

531590

In [17]:
# P-041
using ShiftedArrays: lag

@chain df_receipts begin
    groupby(:sales_ymd)
    combine(:amount => sum)
    sort(:sales_ymd)
    transform(:sales_ymd => lag)
    leftjoin(_, _, on= :sales_ymd_lag => :sales_ymd,
        matchmissing=:equal,
        makeunique=true
    )
    select(:sales_ymd, :amount_sum, :sales_ymd_lag,
        :amount_sum_1 => :amount_sum_lag)
    @rtransform(:diff = :amount_sum - :amount_sum_lag)
    sort(:sales_ymd)
    first(10)
end

Row,sales_ymd,amount_sum,sales_ymd_lag,amount_sum_lag,diff
,Int64,Int64,Int64?,Int64?,Int64?
1,20170101,33723,missing,missing,missing
2,20170102,24165,20170101,33723,-9558
3,20170103,27503,20170102,24165,3338
4,20170104,36165,20170103,27503,8662
5,20170105,37830,20170104,36165,1665
6,20170106,32387,20170105,37830,-5443
7,20170107,23415,20170106,32387,-8972
8,20170108,24737,20170107,23415,1322
9,20170109,26718,20170108,24737,1981


In [18]:
# P-042
daily_amounts = @chain df_receipts begin
    groupby(:sales_ymd)
    combine(:amount => sum)
    sort(:sales_ymd)
end
base = DataFrame(daily_amounts)

for i in 1:3
    column = Symbol("sales_ymd_lag$i")
    transform!(daily_amounts, :sales_ymd => (x -> lag(x, i)) => column)
    leftjoin!(daily_amounts, base, on= column => :sales_ymd,
        matchmissing=:equal,
        makeunique=true
    )
end

@chain daily_amounts begin
    select(:sales_ymd, :amount_sum,
        :amount_sum_1 => :amount_sum_lag1,
        :amount_sum_2 => :amount_sum_lag2,
        :amount_sum_3 => :amount_sum_lag3,
    )
end

Row,sales_ymd,amount_sum,amount_sum_lag1,amount_sum_lag2,amount_sum_lag3
,Int64,Int64,Int64?,Int64?,Int64?
1,20170101,33723,missing,missing,missing
2,20170102,24165,33723,missing,missing
3,20170103,27503,24165,33723,missing
4,20170104,36165,27503,24165,33723
5,20170105,37830,36165,27503,24165
6,20170106,32387,37830,36165,27503
7,20170107,23415,32387,37830,36165
8,20170108,24737,23415,32387,37830
9,20170109,26718,24737,23415,32387


In [19]:
# P-043 年代・性別ごとの売上金額合計
@chain df_receipts begin
    innerjoin(df_customers, 
        on= :customer_id => :customer_id,
        makeunique=true)
    @rtransform(:generation = floor(Int, :age / 10) * 10)
    groupby([:gender_cd, :generation])
    combine(:amount => sum)
    unstack(:generation, :gender_cd, :amount_sum)
    rename([:generation, :male, :female, :unknown])
end

Row,generation,male,female,unknown
,Int64,Int64?,Int64?,Int64?
1,10,1591,149836,4317
2,20,72940,1363724,44328
3,30,177322,693047,50441
4,40,19355,9320791,483512
5,50,54320,6685192,342923
6,60,272469,987741,71418
7,70,13435,29764,2427
8,80,46360,262923,5111
9,90,missing,6260,missing


In [20]:
# P-044 年代、性別コード、売上金額の縦持ち。コード値は0埋め
@chain df_receipts begin
    innerjoin(df_customers, on= :customer_id => :customer_id,
        makeunique=true)
    @rtransform(:generation = floor(Int, :age / 10) * 10)
    @rtransform(:gender_cd = string(:gender_cd, pad = 2))
    groupby([:generation, :gender_cd])
    combine(:amount => sum)
    sort([:generation, :gender_cd])
end

Row,generation,gender_cd,amount_sum
,Int64,String,Int64
1,10,00,1591
2,10,01,149836
3,10,09,4317
4,20,00,72940
5,20,01,1363724
6,20,09,44328
7,30,00,177322
8,30,01,693047
9,30,09,50441


In [21]:
# P-045 日付を文字列に変換
using Dates
@chain df_customers begin
    @rtransform(:birth_day = Dates.format(:birth_day, "YYYYmmdd"))
    select(:customer_id, :birth_day)
    first(10)
end

Row,customer_id,birth_day
,String15,String
1,CS021313000114,19810429
2,CS037613000071,19520401
3,CS031415000172,19761004
4,CS028811000001,19330327
5,CS001215000145,19950329
6,CS020401000016,19740915
7,CS015414000103,19770809
8,CS029403000008,19730817
9,CS015804000004,19310502


In [22]:
# P-046 数値を日付型に変換
@chain df_customers begin
    @rtransform(:application_date = Date(string(:application_date), "YYYYmmdd"))
    select(:customer_id, :application_date)
    first(10)
end

Row,customer_id,application_date
,String15,Date
1,CS021313000114,2015-09-05
2,CS037613000071,2015-04-14
3,CS031415000172,2015-05-29
4,CS028811000001,2016-01-15
5,CS001215000145,2017-06-05
6,CS020401000016,2015-02-25
7,CS015414000103,2015-07-22
8,CS029403000008,2015-05-15
9,CS015804000004,2015-06-07


In [23]:
# P-047 
@chain df_receipts begin
    @rtransform(:sales_ymd = Date(string(:sales_ymd), "YYYYmmdd"))
    select(:receipt_no, :receipt_sub_no, :sales_ymd)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_ymd
,Int64,Int64,Date
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


In [24]:
# P-048
using Dates
@chain df_receipts begin
    @rtransform(:sales_date = Date(unix2datetime(:sales_epoch)))
    select(:receipt_no, :receipt_sub_no, :sales_date)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_date
,Int64,Int64,Date
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


In [25]:
# P-049
using Dates
@chain df_receipts begin
    @rtransform(:sales_date = Date(unix2datetime(:sales_epoch)))
    @rtransform(:sales_year = Dates.year(:sales_date))
    select(:receipt_no, :receipt_sub_no, :sales_year)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_year
,Int64,Int64,Int64
1,112,1,2018
2,1132,2,2018
3,1102,1,2017
4,1132,1,2019
5,1102,2,2018
6,1112,1,2019
7,1102,2,2018
8,1102,1,2019
9,1112,2,2017


In [26]:
# P-050
using Dates
@chain df_receipts begin
    @rtransform(:sales_date = unix2datetime(:sales_epoch))
    @rtransform(:sales_month = string(Dates.month(:sales_date), pad = 2))
    select(:receipt_no, :receipt_sub_no, :sales_month)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_month
,Int64,Int64,String
1,112,1,11
2,1132,2,11
3,1102,1,07
4,1132,1,02
5,1102,2,08
6,1112,1,06
7,1102,2,12
8,1102,1,09
9,1112,2,05
